In [4]:
import os
import sys
import tqdm
import math
# import pygrib
import pickle
import argparse
import datetime
import time
import numpy as np
import pandas as pd
import xarray as xr
from scipy import interpolate
from dateutil.relativedelta import relativedelta
import tqdm
import math

## 读取观测站并固定华东站点

In [5]:
os.chdir('/mnt/pami14/DATASET/METEOROLOGY/GT_micaps')
## select fixed 3 micaps file

#1. 选定站点数多的几个时间戳，否则做时序特征提取的时候没法统一序列~~

"""
结论：除了08、14和20时刻，其他时刻的站点数都少于1000，然后不同时刻的站点不是全相交的，并不是
完全的包含关系~~

一般交集在之前最少的站点的基础之上，再少一点，如果多个少站点时次相交也亦然~~
"""

# 最大站点数 02  
east_locBound = (20, 40, 110, 124)
# 2147 ***
candi_mi_path1 = os.path.join("2017", "08", "surface", "r6-p", "17080608.000")
# 2407 ***
candi_mi_path2 = os.path.join( "2017", "01", "surface", "r6-p", "17010802.000")
# 835  
# candi_mi_path3 = os.path.join("2017", "01", "surface", "r6-p", "17010802.000")
candi_mi_path3 = os.path.join("2017", "01", "surface", "r6-p", "17010817.000")


# 718
candi_mi_path4 = os.path.join("2017", "01", "surface", "r6-p", "17010805.000")
# 772
candi_mi_path5 = os.path.join("2017", "01", "surface", "r6-p", "17010817.000")
# 2400 ***
candi_mi_path6 = os.path.join("2017", "01", "surface", "r6-p", "17010820.000")
# 835 
candi_mi_path7 = os.path.join("2017", "01", "surface", "r6-p", "17010802.000")

candi_paths_l = [candi_mi_path1, candi_mi_path2, candi_mi_path3]

# init 
index_l = []
index_v = []
lats_d = {}
lons_d = {}

'''
读取观测站
'''
for cPath in candi_paths_l:
    
    index_temp = []

    if not os.path.exists(cPath):
        print("candidate micaps not exists!")

    with open(cPath, encoding="GBK") as f:  
        cPath_data = f.readlines()
        # 从14行开始取
        key_data = cPath_data[14:]
        print ("total_station_number:", len(key_data))
#         print ("content:", key_data)

    for oneLine in key_data:
#         print ('sL:', oneLine)
        oneLabel = oneLine.split()
#         print ('sB:', oneLabel)
        index, lon, lat, _, _ = map(float, oneLabel) 

        # select strictly
        if (lat < east_locBound[0]) or (lat > east_locBound[1]):
            continue
        elif (lon < east_locBound[2]) or (lon > east_locBound[3]):
            continue
        else:
            index_l.append(index)
            index_temp.append(index)
            lats_d[index] = lat        
            lons_d[index] = lon
            
    index_v.append(index_temp)

# remove redundancy and fixed indice      
index_fixed = np.unique(np.array(index_l))


# 计算两个不同时刻站点之间的交集是否完全相较
index_v1 = index_v[0]
index_v2 = index_v[1]
index_v3 = index_v[2]

print (len(index_v1))
print (len(index_v2))
print (len(index_v3))

## 计算交集
index_v1_v2 = [v11 for v11 in index_v1 for v22 in index_v2 if v11==v22]
index_v1_v3 = [v33 for v11 in index_v1 for v33 in index_v3 if v11==v33]
index_v2_v3 = [v22 for v22 in index_v2 for v33 in index_v3 if v22==v33]

# print ('--'*16)

# print (len(index_v1_v2))
# print (len(index_v1_v3))
# print (len(index_v2_v3))


# print (index_v1_v2)

print ('--'*16)

index_v1_v2 = [v11 for v11 in index_v1 for v22 in index_v2 if v11==v22]
print (len(index_v1_v2))
index_v1_v2_v3 = [v33 for v12 in index_v1_v2 for v33 in index_v3 if v12==v33]

# print (index_v1_v2_v3)
print (len(index_v1_v2_v3))


  

total_station_number: 2147
total_station_number: 835
total_station_number: 772
1101
327
308
--------------------------------
300
277


In [70]:
os.chdir('/mnt/pami14/DATASET/METEOROLOGY/micaps_16_18')
## select fixed 3 micaps file
#1. 选定站点数多的几个时间戳，否则做时序特征提取的时候没法统一序列~~

"""
① 0-24共25个量或0-24个量，我截取的单个数据的长度为26

② 

③ 

结论：


"""

candi_mi_path1 = os.path.join("2017", "01", "surface", "r6-p", "17010802.000")

candi_mi_path2 = os.path.join("2017", "01", "surface", "r6-p", "17010802.000")

candi_mi_path3 = os.path.join("2017", "01", "surface", "r6-p", "17010802.000")



# 最大站点数 02  
east_locBound = (20, 40, 110, 124)
# 2147 ***
candi_modal_path1 = os.path.join("2017", "08", "surface", "plot", "17080608.000")
# 2407 ***
candi_modal_path2 = os.path.join( "2017", "07", "surface", "plot", "17070814.000")

candi_modal_path3 = os.path.join("2017", "01", "surface", "plot", "17010802.000")

# 718
candi_modal_path4 = os.path.join("2017", "01", "surface", "plot", "17010805.000")
# 772
candi_modal_path5 = os.path.join("2017", "01", "surface", "plot", "17010817.000")
# 2400 ***
candi_modal_path6 = os.path.join("2017", "01", "surface", "plot", "17010820.000")
# 835 
candi_modal_path7 = os.path.join("2017", "01", "surface", "plot", "17010802.000")

# candi_modals_l = [candi_modal_path1, candi_modal_path2, candi_modal_path3]

candi_modals_l = [candi_modal_path1]

# init 
index_l = []
modals_l = []
lats_d = {}
lons_d = {}


for cPath in candi_modals_l:

    if not os.path.exists(cPath):
        print("candidate modals not exists!")

    with open(cPath, encoding="GBK") as f:
        
        cPath_data = f.readlines()        
        key_data = cPath_data[2:]
#         print ("CONTENT:", key_data)  

    for i, oneLine in enumerate(key_data):
        
        # 截取2段部分
        
        if (i+1)%2==0:
            
            post_label = oneLine.split()                    
            oneLabel = pre_label + post_label  
#             print (oneLabel)
#             print ('modal_lenth:', len(oneLabel))
                        
            lon = float(oneLabel[1])
            lat = float(oneLabel[2])
            
#             print (lon)
#             print (lat)

            if (lat < east_locBound[0]) or (lat > east_locBound[1]):
                continue
            elif (lon < east_locBound[2]) or (lon > east_locBound[3]):
                continue
            else:
                
                index = int(oneLabel[0])
                index_l.append(index)                       
                lats_d[index] = lat        
                lons_d[index] = lon
                                    
                wind_direct = oneLabel[6]
                wind_velocity = oneLabel[7]
                pressure = oneLabel[8]
                
                # humidness
                dew_point = oneLabel[16]
                temperature = oneLabel[19]
            
                modals_l.append((wind_direct, wind_velocity, pressure, dew_point, temperature))
        
        pre_label = oneLine.split()
        
        
print (len(index_l))
# print ('INDICE:', index_l)

# wind_direct, wind_velocity, pressure, dew_point, temperature
print ('MODALS:', modals_l) 




"""

[('180', '2', '34', '28.4', '29.8'), 
('320', '1', '73', '26.0', '28.0'), 
('290', '1', '52', '24.4', '29.3'), 
('200', '2', '44', '26.9', '28.7'), 
('320', '1', '48', '25.6', '29.2'), 
('110', '1', '58', '26.6', '29.8'), 
('220', '2', '62', '25.0', '30.1'), 
('220', '2', '46', '27.2', '29.6'), 
('0', '0', '47', '26.3', '30.6'), 
('220', '2', '38', '27.6', '29.8')]

"""

        
# index_fixed = np.unique(np.array(index_l))


"""fixed_station_number: 1185  17080614.000  17070823.000  16081017.000"""

"""fixed_station_number: 1188  17080608.000  17070814.000  16081020.000"""


 

1194
MODALS: [('180', '2', '34', '28.4', '29.8'), ('320', '1', '73', '26.0', '28.0'), ('290', '1', '52', '24.4', '29.3'), ('200', '2', '44', '26.9', '28.7'), ('320', '1', '48', '25.6', '29.2'), ('110', '1', '58', '26.6', '29.8'), ('220', '2', '62', '25.0', '30.1'), ('220', '2', '46', '27.2', '29.6'), ('0', '0', '47', '26.3', '30.6'), ('220', '2', '38', '27.6', '29.8'), ('180', '1', '44', '26.3', '29.7'), ('250', '2', '40', '27.5', '31.4'), ('40', '1', '49', '27.2', '31.7'), ('200', '1', '49', '25.6', '31.8'), ('110', '1', '50', '25.4', '31.3'), ('220', '2', '55', '24.8', '28.6'), ('180', '1', '47', '26.5', '33.2'), ('140', '4', '15', '23.4', '25.7'), ('90', '2', '13', '24.2', '26.7'), ('220', '1', '15', '28.2', '29.3'), ('180', '2', '25', '27.7', '29.9'), ('20', '1', '15', '26.2', '28.6'), ('200', '2', '15', '26.6', '30.0'), ('290', '1', '61', '17.6', '22.7'), ('360', '3', '61', '14.6', '20.5'), ('340', '1', '22', '24.0', '28.6'), ('90', '1', '16', '22.6', '29.6'), ('270', '1', '69', '

'fixed_station_number: 1188  17080608.000  17070814.000  16081020.000'

In [77]:
os.chdir('/mnt/pami14/DATASET/METEOROLOGY/micaps_16_18')
## select fixed 3 micaps file

#1. 选定站点数多的几个时间戳，否则做时序特征提取的时候没法统一序列~~

# 最大站点数 02  
east_locBound = (20, 40, 110, 124)

"""
① 做数据表明~~同一个时次，diamond1的数据的站点数可以完全包含diamond3的全部站点，所以说

② 选取共同站点，只考虑降水类数据的站点的共同站点就可以，降水有的，diamond1一定有

③ 

结论：


"""

# candi_mi_path1 = os.path.join("2017", "01", "surface", "r6-p", "17010820.000")

# candi_mi_path1 = os.path.join("2017", "08", "surface", "r6-p", "17080608.000")

# candi_mi_path1 = os.path.join("2017", "07", "surface", "r6-p", "17070814.000")


candi_mi_path2 = os.path.join("2017", "01", "surface", "r6-p", "17010802.000")

candi_mi_path3 = os.path.join("2017", "01", "surface", "r6-p", "17010802.000")



# 2147 ***
# candi_modal_path1 = os.path.join("2017", "08", "surface", "plot", "17080608.000")
# candi_modal_path1 = os.path.join("2017", "01", "surface", "plot", "17010820.000")
# candi_modal_path1 = os.path.join( "2017", "07", "surface", "plot", "17070814.000")

# 2407 ***
# candi_modal_path2 = os.path.join( "2017", "07", "surface", "plot", "17070814.000")

candi_modal_path3 = os.path.join("2017", "01", "surface", "plot", "17010802.000")

# 718
candi_modal_path4 = os.path.join("2017", "01", "surface", "plot", "17010805.000")
# 772
candi_modal_path5 = os.path.join("2017", "01", "surface", "plot", "17010817.000")
# 2400 ***
candi_modal_path6 = os.path.join("2017", "01", "surface", "plot", "17010820.000")
# 835 
candi_modal_path7 = os.path.join("2017", "01", "surface", "plot", "17010802.000")

# candi_modals_l = [candi_modal_path1, candi_modal_path2, candi_modal_path3]

candi_modals_l = [candi_modal_path3]
candi_mi_l = [candi_mi_path3]

# init 
index_modal_l = []
index_l = []


print ('--'*16)

for cPath in candi_modals_l:

    if not os.path.exists(cPath):
        print("candidate modals not exists!")

    with open(cPath, encoding="GBK") as f:
        
        cPath_data = f.readlines()        
        key_data = cPath_data[2:]
#         print (key_data)

    for i, oneLine in enumerate(key_data):
                
        if (i+1)%2==0:
            
            post_label = oneLine.split()                    
            oneLabel = pre_label + post_label  
#             print (oneLabel)
#             print ('modal_lenth:', len(oneLabel))
                        
            lon = float(oneLabel[1])
            lat = float(oneLabel[2])

            if (lat < east_locBound[0]) or (lat > east_locBound[1]):
                continue
            elif (lon < east_locBound[2]) or (lon > east_locBound[3]):
                continue
            else:
                index = int(oneLabel[0])
                index_modal_l.append(index) 
                
                
        pre_label = oneLine.split()
            
        
print (len(index_modal_l))




for cPath in candi_mi_l:
    

    if not os.path.exists(cPath):
        print("candidate micaps not exists!")

    with open(cPath, encoding="GBK") as f:  
        cPath_data = f.readlines()
        key_data = cPath_data[14:]
        

    for oneLine in key_data:
        oneLabel = oneLine.split()
        index, lon, lat, _, _ = map(float, oneLabel) 

        # select strictly
        if (lat < east_locBound[0]) or (lat > east_locBound[1]):
            continue
        elif (lon < east_locBound[2]) or (lon > east_locBound[3]):
            continue
        else:
            index_l.append(index)
            
print (len(index_l))



index_v1_v2 = [modIdx for modIdx in index_modal_l for idx in index_l if modIdx==idx]

print (len(index_v1_v2))


 

--------------------------------
340
327
327


## fix updated stations items testing 

## scope sliding Windows testing 

In [6]:
os.chdir('/mnt/pami14/DATASET/METEOROLOGY/GT_micaps')
GT_micaps_path = "/mnt/pami14/DATASET/METEOROLOGY/GT_micaps"
utility_path = "/mnt/pami14/yqliu/biasCorrection/utility_dataset"

fixstation_update1_path = os.path.join(utility_path, "stationItem.npy")

east_locBound = (20, 40, 110, 124)


## test I  2017

## 00 + 8 *** n = 1101 ✔️
candi_mi_path1 = os.path.join("2017", "08", "surface", "r6-p", "17080608.000")
## 06 + 8  *** n = 1101 ✔️
candi_mi_path2 = os.path.join( "2017", "08", "surface", "r6-p", "17080814.000")
## 9 + 8  *** n = 226 ✘
candi_mi_path3 = os.path.join("2017", "08", "surface", "r6-p", "17080817.000")
## 12 + 8  ***  n = 1098 ✔️
candi_mi_path4 = os.path.join("2017", "08", "surface", "r6-p", "17080820.000")
## 18 + 8  *** n = 300 ✘
candi_mi_path5 = os.path.join("2017", "08", "surface", "r6-p", "17080902.000")
## 21 + 8  *** n = 231 ✘
candi_mi_path6 = os.path.join("2017", "08", "surface", "r6-p", "17080905.000")


"""
# ## test II  2016
## 00 + 8 *** n = 1175  ✔️
candi_mi_path1 = os.path.join("2016", "12", "surface", "r6-p", "16120608.000")
## 06 + 8  *** n = 1175 ✔️
candi_mi_path2 = os.path.join( "2016", "12", "surface", "r6-p", "16120614.000")
## 9 + 8  *** n = 327  ✘
candi_mi_path3 = os.path.join("2016", "12", "surface", "r6-p", "16120617.000")
## 12 + 8  ***  n = 1174 ✔️
candi_mi_path4 = os.path.join("2016", "12", "surface", "r6-p", "16120620.000")
## 18 + 8  *** n = 316  ✘
candi_mi_path5 = os.path.join("2016", "12", "surface", "r6-p", "16120702.000")
## 21 + 8  *** n =  5  ✘
candi_mi_path6 = os.path.join("2016", "12", "surface", "r6-p", "16120705.000")
"""


candi_paths_l = [candi_mi_path1, candi_mi_path2, candi_mi_path3, candi_mi_path4, candi_mi_path5, candi_mi_path6]

candi_paths_l = candi_paths_l[0:2] 
# + [candi_paths_l[3], candi_paths_l[5]]

#  

# init 
update2_stationId_l = []
update2_item_l = []
update2_origin_indice_l = []

index_total = []
lats_d = {}
lons_d = {}
candi_no = len(candi_paths_l)

# load updated fixed stations and get id
latLonItem = np.load(fixstation_update1_path)

    
station_update1_id_l = [latLonItem[k][0] for k, item in enumerate(latLonItem)]
print (len(station_update1_id_l))
print ('-'*16)

for cPath in candi_paths_l:

    if not os.path.exists(cPath):
        print("candidate micaps not exists!")

    with open(cPath, encoding="GBK") as f:  
        cPath_data = f.readlines()
        key_data = cPath_data[14:]
        print ("total_station_number:", len(key_data))

    for oneLine in key_data:
        oneLabel = oneLine.split()
        index, lon, lat, _, _ = map(float, oneLabel) 

        # select strictly
        if (lat < east_locBound[0]) or (lat > east_locBound[1]):
            continue
        elif (lon < east_locBound[2]) or (lon > east_locBound[3]):
            continue
        else:
            # total stationId
            index_total.append(index)
            # all lats and lons
            lats_d[index] = lat        
            lons_d[index] = lon
            

# count station occurrence number and return update stationID list
for originIdx, stationId in enumerate(station_update1_id_l):
    
    if index_total.count(stationId) == candi_no:
        update2_stationId_l.append(stationId)
        update2_origin_indice_l.append(originIdx)
        update2_item_l.append((stationId, lats_d[stationId], lons_d[stationId]))
        
        
# print ('update2_stationIds_length:', len(update2_stationId_l))
# print (update2_origin_indice_l)
print (update2_stationId_l[0])
print (lats_d[update2_stationId_l[0]])
print (lons_d[update2_stationId_l[0]])


print ('-'*16)





1188
----------------
total_station_number: 2147
total_station_number: 2404
45005.0
22.3
114.17
----------------


In [6]:
"""generate "EC_Feature - shape: 180 * 74 * 193 * 145"""
ec_Features = np.random.rand(180, 74, 193, 145)
east_locBound = (20, 40, 110, 124)
sw_Bound_lat = np.arange(east_locBound[0] -2 ,east_locBound[1] + 2, 0.125)[::-1]
sw_Bound_lon = np.arange(east_locBound[2] -2 ,east_locBound[3] + 2, 0.125)

scopeSW = 0.125
gw_range = math.floor(18 / 2)

ec_features_stations_l = []
update_lats_lons_item = []
update_indice = []

## save bias info
bias_info = []

before_time = time.time()

for k in range(len(lats_lons_item)):
    flag = False
#     print ('0---0')
    for i, lat in enumerate(sw_Bound_lat):
        if flag == True:
#             print ('1---1')
            break
#         print ('2---2')
        for j, lon in enumerate(sw_Bound_lon):

            lat_range = (lat - scopeSW, lat)
            lon_range = (lon, lon + scopeSW)
#             print ('3---3')
#             print ('lat:', lats_lons_item[k][1])
#             print ('latRange:', (lat_range[0], lat_range[1]))
            
    
            if lats_lons_item[k][1] <= lat_range[1] and lats_lons_item[k][1]>= lat_range[0] and lats_lons_item[k][2] <= lon_range[1] and lats_lons_item[k][2]>= lon_range[0]:
                
#                 print ('---')
                
                
                bias_info.append((abs(lat - lats_lons_item[k][1]), abs(lon - lats_lons_item[k][2])))
                  
                ec_features_stations_l.append(ec_Features[:,:,(i - gw_range):(i + gw_range + 1), (j - gw_range):(j + gw_range + 1)])
                update_lats_lons_item.append((lats_lons_item[k][0], lats_lons_item[k][1], lats_lons_item[k][2]))
                update_indice.append(lats_lons_item[k][0])

                flag = True
                break
                
after_time = time.time()

print ("cost_time:", (after_time - before_time))
                
# list → numpy n * N * C * h * w        
# print (ec_features_stations_l)
print (type(ec_features_stations_l))

 # n * N * C * h * w 
ec_features_stations = np.stack(ec_features_stations_l)
update_latlons_item = update_lats_lons_item

print ('EC_Features_keyStations_shape:', ec_features_stations.shape)
# print ('update_latlons_keyStations:', update_latlons_item)
# print ("update_station_number:", lats_lons_item[k][0])


# print ("latlon_bias_info:", bias_info)

"""lat/lon bias <= 0.125"""


NameError: name 'lats_lons_item' is not defined

## 读取EC模式数据

In [3]:
import pickle
import os
import time
import numpy as np


os.chdir('/mnt/pami14/yqliu/EC-BiasCorrecting/SHO/1617_benchmarkEC/0-ec201608')
pkl_name = os.path.join("2016082612_ec.pkl")

ec_tsFeatures_l = []
ec_tsStations_l = []
es_tsFS = []



if not os.path.exists(pkl_name):
    raise NameError("No %s file exists." % (pkl_name))

start_time = time.time()
# with open(pkl_name, 'rb') as f:
#     print ("stating...")
#     oneCropData = pickle.load(f)


with open(pkl_name, 'rb') as f:
    print ("stating...")
    oneCropData = np.load(f)


print (type(oneCropData))
    
print ("cost_time:", time.time() - start_time)

# print (oneCropData.values)
# n * N * C * h * w 

for arr in (oneCropData.values()):
    
    oneCropData = arr.values
    
    
    
print (oneCropData)
print (type(oneCropData))

print ('--'*10)

print (arr.coords["key_channel"])
print (arr.coords["seq_length"])
print (arr.coords["lat_range"])
print (arr.coords["lon_range"])

print ('--'*10)

strpath = arr.attrs["prep_gt"]
strpath = arr.attrs["fix_stationIndice"]


print ('--'*10)






stating...
<class 'dict'>
cost_time: 0.9719581604003906
[[[6.8000000e+01 7.0500000e+01 7.3000000e+01 ... 6.8500000e+01
   6.9750000e+01 7.1000000e+01]
  [6.8750000e+01 6.9500000e+01 7.0250000e+01 ... 6.7000000e+01
   6.8375000e+01 6.9750000e+01]
  [6.9500000e+01 6.8500000e+01 6.7500000e+01 ... 6.5500000e+01
   6.7000000e+01 6.8500000e+01]
  ...
  [3.3000000e+01 3.1750000e+01 3.0500000e+01 ... 4.3000000e+01
   4.3500000e+01 4.4000000e+01]
  [3.2000000e+01 3.1125000e+01 3.0250000e+01 ... 3.8500000e+01
   3.9500000e+01 4.0500000e+01]
  [3.1000000e+01 3.0500000e+01 3.0000000e+01 ... 3.4000000e+01
   3.5500000e+01 3.7000000e+01]]

 [[7.0189598e+01 6.9189598e+01 6.8189598e+01 ... 8.5189598e+01
   8.7439598e+01 8.9689598e+01]
  [6.9689598e+01 7.0064598e+01 7.0439598e+01 ... 8.3439598e+01
   8.5189598e+01 8.6939598e+01]
  [6.9189598e+01 7.0939598e+01 7.2689598e+01 ... 8.1689598e+01
   8.2939598e+01 8.4189598e+01]
  ...
  [5.4189602e+01 5.4689602e+01 5.5189602e+01 ... 4.7189602e+01
   5.0689602

KeyError: 'seq_length'

In [ ]:
os.chdir('/home/yqliu/Yiqun_bias/demo_save')

l = os.listdir(os.path.join("test_osdir"))

print (l)

l==[]


In [ ]:
import os
import numpy as np
import torch
from torchvision import transforms

print (torch.__version__)


os.chdir('/mnt/pami14/yqliu/biasCorrection/utility_dataset')

meanFilePath = os.path.join( "mean.npy")
stdFilePath = os.path.join("std.npy")

mean_array = np.load(meanFilePath)
std_array = np.load(stdFilePath)

print (mean_array.shape)

print (std_array.shape)


print ('meanArray:', mean_array)
print ('stdArray:', std_array)


# z-score
transformer = transforms.Compose([
    transforms.Normalize(mean = mean_array, std = std_array)
])



#### CHANNEL NORMALIZATION  57 * 6 * 19 * 19
random_f = np.random.rand(57,6,19,19)

print (random_f.shape)

print ('random_f:', random_f)

d_feature = torch.from_numpy(random_f.astype(np.float32))

# print ('d_feature:', d_feature)

print (d_feature.size())

dfeatures_1 = d_feature.view(57, 6 * 19, 19)

print ('dfeatures_1_shape:', dfeatures_1.shape)

# random_temp = torch.from_numpy(np.random.rand(37,17,17).astype(np.float32))

# print ('random_temp:', random_temp)

nor_features = transformer(dfeatures_1)
print ('features_nor_shape:', nor_features.shape)

Features = nor_features.view(57, 6, 19, 19)
print ('Features_shape:', Features.shape)



print ('Features:', Features)

print ('Features_type:', type(Features))









In [9]:
os.chdir('/mnt/pami14/yqliu/EC-BiasCorrecting/SHO/1617_benchmarkEC/0-ec201608')

pkl_name = os.path.join("2016080106_ec" +".pkl")

x_l = []

print (pkl_name)

if not os.path.exists(pkl_name):
    raise NameError("No %s file exists." % (pkl_name))

with open(pkl_name, 'rb') as f:
    oneCropData = pickle.load(f)
    
    
for f in oneCropData.values():
    
    x_l.append(f)
    

x_l_xar = x_l[3]

xar_f = x_l_xar.values


print (xar_f.shape)
    


2016080106_ec.pkl
(57, 29, 29)
